In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json
from tqdm import tqdm

In [219]:
def get_topic_url(soup):
    topic_urls = []
    for i, title in enumerate(soup.find_all('div',{'class':'category--header'})):
        icon = True if title.find('i') else False
        if not icon:
            if int(title.find('span',{'class':'topic_number'}).text) > 5:
                topic_urls.append(
                    {
                    'title':title.find('a').text.strip(),
                    'url':title.find('a')['href'].strip()
                }
                )
    return topic_urls

In [255]:
with open('topics/shop-cloths.jsonl', 'a', encoding="utf-8") as outfile:
    for n in tqdm(range(1+681+405, 1155)):
        category_topics_url_string = 'https://www.ninisite.com/discussion/forum/89/%d9%be%d9%88%d8%b4%d8%a7%da%a9'
        category_topics_url = urlopen(category_topics_url_string+f'?page={n}')
        category_topics = category_topics_url.read()
        category_topics_soup = BeautifulSoup(category_topics, 'html.parser')
        topic_urls = get_topic_url(category_topics_soup)
        for url in topic_urls:
            json.dump(url, outfile, ensure_ascii=False)
            outfile.write('\n')

100%|██████████| 68/68 [03:30<00:00,  3.10s/it]


In [30]:
with open('topics/shop-toys.jsonl') as json_file:
    data = json_file.readlines()
    data = [d.strip() for d in data]

In [37]:
topic_url_string ='https://ninisite.com/'+eval(data[0])['url']
topic_url = urlopen(topic_url_string+'?page=1')
topic_content = topic_url.read()
topic_soup = BeautifulSoup(topic_content, 'html.parser')
pagination = topic_soup.find_all('a',{'class':'page-link'}) 
pages = 1 if len(pagination) == 0 else int(pagination[-2].text)

In [39]:
conv = []
for p in tqdm(range(1,pages+1)):
    if pages > 1:
        topic_url = urlopen(topic_url_string+f'?page={p}')
        topic_content = topic_url.read()
        topic_soup = BeautifulSoup(topic_content, 'html.parser')

    for i, article in enumerate(topic_soup.find_all('article')):
        message = [p.text for p in article.find('div',{'class':'post-message'}).find_all('p')]

        reply_message_div = article.find('div',{'class':'reply-message'})
        reply_message_text = reply_message_div.text if reply_message_div else None
        
        if reply_message_text != None:
            if len(reply_message_text)  < 90:
                reply_message = reply_message_text
            else:
                reply_message_id = reply_message_div['data-id'] if reply_message_div else None
                
                if reply_message_id != None:
                    reply_message = topic_soup.find('article', attrs={'id':f'post-{reply_message_id}'})
                    
                    if reply_message != None:
                        reply_message = [p.text for p in reply_message.find('div',{'class':'post-message'}).find_all('p')]
                    else:
                        reply_message_url_string = topic_url_string+f'?postId={reply_message_id}'
                        reply_message_url = urlopen(reply_message_url_string)
                        reply_message_content = reply_message_url.read()
                        reply_message_soup = BeautifulSoup(reply_message_content, 'html.parser')
                        reply_message = reply_message_soup.find('article', attrs={'id':f'post-{reply_message_id}'})
                        reply_message = [p.text for p in reply_message.find('div',{'class':'post-message'}).find_all('p')]
        else:
            reply_message = topic_soup.find('article', attrs={'id':'topic'})
            reply_message_title = topic_soup.find('article', attrs={'id':'topic'}).find('h1',{'class':'topic-title'}).find('a').text
            reply_message = [p.text for p in reply_message.find('div',{'class':'post-message'}).find_all('p')]

        if i != 0:
            conv.append(
                {
                    "topic": reply_message_title,
                    "question":reply_message,
                    "answer":message,
                }
            )

100%|██████████| 45/45 [02:27<00:00,  3.28s/it]


In [3]:
import os
categories_url_path = os.listdir('topics')

all_topics = []
for cat in categories_url_path:
    with open(f'topics/{cat}') as jsonl:
        topics = jsonl.readlines()
        for d in topics:
            all_topics.append(d.strip())


In [10]:
from os.path import isfile, join
from os import listdir

def get_urls(path):
    categories_url_path =  [f for f in listdir(path) if isfile(join(path, f))]
    all_topics = []
    for cat in categories_url_path:
        with open(f'topics/{cat}') as jsonl:
            topics = jsonl.readlines()
            for d in topics:
                all_topics.append((cat, d.strip()))
    return list(set(all_topics))

def get_crawled(path):
    with open(path) as jsonl:
        craweld_topics = jsonl.readlines()
        craweld_topics = [t.strip() for t in craweld_topics]
    return craweld_topics

In [11]:
all_topics = get_urls('topics')
craweld_topics = get_crawled('crawled_topics.jsonl')


In [13]:
craweld_topics

['{"title": "تو رو خدا کمکم کنید ، امروز رفتم سونو بهم گفته که سرخرگ بند ناف جنین من یکی هستش", "url": "/discussion/topic/20450/%d8%aa%d9%88-%d8%b1%d9%88-%d8%ae%d8%af%d8%a7-%da%a9%d9%85%da%a9%d9%85-%da%a9%d9%86%db%8c%d8%af-%d8%a7%d9%85%d8%b1%d9%88%d8%b2-%d8%b1%d9%81%d8%aa%d9%85-%d8%b3%d9%88%d9%86%d9%88-%d8%a8%d9%87%d9%85-%da%af%d9%81%d8%aa%d9%87-%da%a9%d9%87-%d8%b3%d8%b1%d8%ae%d8%b1%da%af-%d8%a8%d9%86%d8%af-%d9%86%d8%a7%d9%81-%d8%ac%d9%86%db%8c%d9%86-%d9%85%d9%86-%db%8c%da%a9%db%8c-%d9%87%d8%b3%d8%aa%d8%b4"}',
 '{"title": "فکر نمیکنم کسی تو دنیا باشه که این همه از زندگیش خسته شده باشه😔", "url": "/discussion/topic/8498130/%d9%81%da%a9%d8%b1-%d9%86%d9%85%db%8c%da%a9%d9%86%d9%85-%da%a9%d8%b3%db%8c-%d8%aa%d9%88-%d8%af%d9%86%db%8c%d8%a7-%d8%a8%d8%a7%d8%b4%d9%87-%da%a9%d9%87-%d8%a7%db%8c%d9%86-%d9%87%d9%85%d9%87-%d8%a7%d8%b2-%d8%b2%d9%86%d8%af%da%af%db%8c%d8%b4-%d8%ae%d8%b3%d8%aa%d9%87-%d8%b4%d8%af%d9%87-%d8%a8%d8%a7%d8%b4%d9%87"}',
 '{"title": "ترشحات زرد رنگ در هفته 30 بارداری.....نگرانم..

In [12]:
all_topics

[('pregnancy-other.jsonl',
  '{"title": "من استرس گرفتم قلبم دردمیکنه", "url": "/discussion/topic/4873317/%d9%85%d9%86-%d8%a7%d8%b3%d8%aa%d8%b1%d8%b3-%da%af%d8%b1%d9%81%d8%aa%d9%85-%d9%82%d9%84%d8%a8%d9%85-%d8%af%d8%b1%d8%af%d9%85%db%8c%da%a9%d9%86%d9%87"}'),
 ('pregnancy-birth.jsonl',
  '{"title": "خیانت و قطع رابطه", "url": "/discussion/topic/6894014/%d8%ae%db%8c%d8%a7%d9%86%d8%aa-%d9%88-%d9%82%d8%b7%d8%b9-%d8%b1%d8%a7%d8%a8%d8%b7%d9%87"}'),
 ('kids-growth.jsonl',
  '{"title": "دکتر اطفال که دخترمو بردم مورد داشت", "url": "/discussion/topic/3306930/%d8%af%da%a9%d8%aa%d8%b1-%d8%a7%d8%b7%d9%81%d8%a7%d9%84-%da%a9%d9%87-%d8%af%d8%ae%d8%aa%d8%b1%d9%85%d9%88-%d8%a8%d8%b1%d8%af%d9%85-%d9%85%d9%88%d8%b1%d8%af-%d8%af%d8%a7%d8%b4%d8%aa"}'),
 ('life-style-fashion.jsonl',
  '{"title": "صفحش روشن نمیشه", "url": "/discussion/topic/7627578/%d8%b5%d9%81%d8%ad%d8%b4-%d8%b1%d9%88%d8%b4%d9%86-%d9%86%d9%85%db%8c%d8%b4%d9%87"}'),
 ('family-solutions.jsonl',
  '{"title": "در اوج ناامیدی چطوری امیدوار شدید

In [137]:
test = 'تنهاس تنهایی بد دردی چقد خانواده شوهرم نیش بهم زدن چقد غربت سخته .بچم با سهل انگار دکترم مرد .......'
len(test)

100

2 is even.
